In [15]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import glob
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import time
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
import gc
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation,Dropout,BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras.optimizers import Adam

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'sample_submission.csv', 'train']


In [2]:
data = '../input/'
train_files = sorted(glob.glob(data + 'train/*/*'))
# test_path = sorted(glob.glob(data + 'test/*'))

labels = ['HTC-1-M7', 'iPhone-4s', 'iPhone-6', 'LG-Nexus-5x', 'Motorola-Droid-Maxx', 
         'Motorola-Nexus-6', 'Motorola-X', 'Samsung-Galaxy-Note3', 'Samsung-Galaxy-S4',
         'Sony-NEX-7'] 
All_labels = [labels.index(file.split('/')[-2]) for file in train_files] # TO BE CHANGED!!!!!!!

x_train_files, x_test_files, y_train_files, y_test_files = train_test_split(train_files,All_labels,test_size = 0.2, shuffle = True,random_state = 42)

y_train = keras.utils.to_categorical(y_train_files, len(labels)) # one hot vector
y_test = keras.utils.to_categorical(y_test_files, len(labels)) # one hot vector

In [3]:
dim = 256

kernel_filter = 1/12. * np.array([\
            [-1,  2,  -2,  2, -1],  \
            [ 2, -6,   8, -6,  2],  \
            [-2,  8, -12,  8, -2],  \
            [ 2, -6,   8, -6,  2],  \
            [-1,  2,  -2,  2, -1]])


train_imgs = []
for file in tqdm_notebook(x_train_files):
    img = cv2.imread(file)
    center_x = img.shape[0]//2
    center_y = img.shape[1]//2 

    cropped_img = img[center_x-dim//2:center_x+dim//2, center_y-dim//2:center_y+dim//2, : ]
    
    cropped_img = cv2.filter2D(cropped_img.astype(np.float32),-1,kernel_filter)
    
    train_imgs.append(cropped_img)
    del file,img,center_x,center_y,cropped_img
    gc.collect()

train_imgs = np.array(train_imgs)

In [6]:
gc.collect()

0

In [23]:
datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip = True)

datagen.fit(train_imgs)

In [24]:
model=Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),strides=(2, 2), input_shape=(256, 256, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(64, kernel_size=(3, 3),strides=(2, 2) ))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(Conv2D(32, kernel_size=(3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(10))
model.add(Activation('softmax'))

opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 127, 127, 64)      1792      
_________________________________________________________________
batch_normalization_16 (Batc (None, 127, 127, 64)      256       
_________________________________________________________________
activation_31 (Activation)   (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 63, 63, 64)        36928     
_________________________________________________________________
batch_normalization_17 (Batc (None, 63, 63, 64)        256       
_________________________________________________________________
activation_32 (Activation)   (None, 63, 63, 64)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 61, 61, 32)        18464     
__________

In [ ]:
model.fit_generator(datagen.flow(train_imgs, y_train, batch_size=32),steps_per_epoch= 2200 / 32, epochs=100) 

In [ ]:
# del train_imgs
gc.collect()

In [4]:
dim = 256

kernel_filter = 1/12. * np.array([\
            [-1,  2,  -2,  2, -1],  \
            [ 2, -6,   8, -6,  2],  \
            [-2,  8, -12,  8, -2],  \
            [ 2, -6,   8, -6,  2],  \
            [-1,  2,  -2,  2, -1]])


test_imgs = []
for file in tqdm_notebook(x_test_files):
    img = cv2.imread(file)
    center_x = img.shape[0]//2
    center_y = img.shape[1]//2 
    cropped_img = img[center_x-dim//2:center_x+dim//2, center_y-dim//2:center_y+dim//2, : ]
    
    cropped_img = cv2.filter2D(cropped_img.astype(np.float32),-1,kernel_filter)    
    test_imgs.append(cropped_img)
    del file,img,center_x,center_y,cropped_img
    gc.collect()

test_imgs = np.array(test_imgs)

In [22]:
scores = model.evaluate(test_imgs,y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

550/550 [==============================] - 1s 2ms/step
Test loss: 0.6678022979606282
Test accuracy: 0.7890909088741649


In [ ]:
del test_imgs
gc.collect()